### Import 

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from spacy.lang.fr import stop_words
from sklearn.linear_model import LogisticRegression

stop_words = list(stop_words.STOP_WORDS)
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import mlflow
import subprocess

### Connect to server and enable MlFlow.sklearn autolog

In [9]:
commit_hash = subprocess.check_output(["git", "rev-parse", "HEAD"])

In [ ]:
remote_server_uri = "http://localhost:5000/"
mlflow.set_tracking_uri(remote_server_uri)
mlflow.sklearn.autolog(log_datasets=False)
mlflow.set_experiment("/1st-experiment")
mlflow.start_run()
mlflow.set_tags({
"mlflow.note.content" : "This is MLOPS project",
"mlflow.source.name" : "model_design_2.ipynb",
"mlflow.source.git.commit" : commit_hash,
"mlflow.source.git.branch" : "HEAD",
})



### Loading data

In [3]:
df_train = pd.read_csv("../../data/train.csv")
df_test = pd.read_csv("../../data/test.csv")
df_valid = pd.read_csv("../../data/valid.csv")

### Manual logging for dataset

### Pipeline

In [4]:
pipe = Pipeline([
    ("tfidf", TfidfVectorizer(stop_words=stop_words)),
    ("logreg", LogisticRegression( solver="liblinear"))
])

pipe.fit(df_train["review"], df_train["polarity"])
y_pred = pipe.predict(df_test["review"])

/home/ainazeaze/Etude/MLOPS/mlops_project/.venv/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(
2025/11/18 15:13:45 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/11/18 15:13:45 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


### Add test accuracy score

In [ ]:
Y_test = df_test["polarity"].to_numpy()
test_accuracy_score = accuracy_score(pipe.predict(df_test["review"]), Y_test)
mlflow.log_metric("test_accuracy_score", test_accuracy_score)

### Close mlflow run

In [6]:
mlflow.end_run()

🏃 View run puzzled-hare-223 at: http://localhost:5000/#/experiments/529200409768567991/runs/1d7562486d204607915c8124878a3f56
🧪 View experiment at: http://localhost:5000/#/experiments/529200409768567991
